<a href="https://colab.research.google.com/github/nizhantiw/Adding-List-Names---Working-with-Fragments-portals-refs-Introducing-React-Por---syx8kfh4cum8/blob/master/Assignment_Pathfinding_with_A_.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
import heapq

# A Node class to store grid cell information
class Node:
    def __init__(self, position, parent=None):
        self.position = position  # (row, col) tuple
        self.parent = parent
        self.g = 0  # Cost from start to current node
        self.h = 0  # Heuristic cost from current node to end
        self.f = 0  # Total cost (g + h)

    # Allow comparison of nodes for the priority queue
    def __lt__(self, other):
        return self.f < other.f

    # Check for equality to avoid duplicates in lists
    def __eq__(self, other):
        return self.position == other.position

def manhattan_distance(pos1, pos2):
    """
    Calculates the Manhattan distance heuristic between two points.
    This heuristic is consistent and admissible for grid-based pathfinding
    with non-diagonal movement.
    """
    return abs(pos1[0] - pos2[0]) + abs(pos1[1] - pos2[1])

def get_movement_cost(grid, position):
    """
    Returns the cost of moving into a given cell.
    - Rough terrain (row 5) costs 3.
    - All other traversable cells cost 1.
    """
    row, col = position
    # Check for rough terrain on row 5
    if row == 5:
        return 3
    return 1

def a_star_search(grid, start_pos, goal_pos):
    """
    Implements the A* pathfinding algorithm.
    - Uses a priority queue (min-heap) for the open list.
    - Uses a set for the closed list for fast lookups.
    """
    start_node = Node(start_pos)
    goal_node = Node(goal_pos)

    # Initialize open and closed lists [cite: 13]
    open_list = []
    heapq.heappush(open_list, start_node)
    closed_list = set()

    nodes_expanded = 0

    while open_list:
        # Get the node with the lowest f-cost from the priority queue
        current_node = heapq.heappop(open_list)
        nodes_expanded += 1

        # If we reached the goal, reconstruct and return the path
        if current_node == goal_node:
            path = []
            cost = current_node.g
            current = current_node
            while current is not None:
                path.append(current.position)
                current = current.parent
            return path[::-1], cost, nodes_expanded  # Return reversed path, cost, and nodes expanded

        # Add the current node to the closed list
        closed_list.add(current_node.position)

        # Explore neighbors
        for move in [(0, 1), (0, -1), (1, 0), (-1, 0)]:  # Right, Left, Down, Up
            neighbor_pos = (current_node.position[0] + move[0], current_node.position[1] + move[1])

            # Check if neighbor is within grid boundaries
            if not (0 <= neighbor_pos[0] < len(grid) and 0 <= neighbor_pos[1] < len(grid[0])):
                continue

            # Check if neighbor is an obstacle
            if grid[neighbor_pos[0]][neighbor_pos[1]] == 1: # 1 represents an obstacle
                continue

            # Check if neighbor is already in the closed list
            if neighbor_pos in closed_list:
                continue

            neighbor_node = Node(neighbor_pos, current_node)

            # Calculate costs for the neighbor [cite: 14]
            move_cost = get_movement_cost(grid, neighbor_pos)
            neighbor_node.g = current_node.g + move_cost
            neighbor_node.h = manhattan_distance(neighbor_node.position, goal_pos)
            neighbor_node.f = neighbor_node.g + neighbor_node.h

            # If neighbor is in open list with a higher g-cost, skip
            if any(open_node for open_node in open_list if neighbor_node == open_node and neighbor_node.g >= open_node.g):
                continue

            # Add neighbor to the open list
            heapq.heappush(open_list, neighbor_node)

    return None, 0, nodes_expanded # No path found

def print_grid_with_path(grid, path):
    """Helper function to visualize the grid and the path."""
    path_set = set(path) if path else set()
    for r_idx, row in enumerate(grid):
        line = ""
        for c_idx, cell in enumerate(row):
            pos = (r_idx, c_idx)
            if pos == path[0]:
                line += " S "
            elif pos == path[-1]:
                line += " G "
            elif pos in path_set:
                line += " * "
            elif cell == 1:
                line += " # "
            elif r_idx == 5:
                line += " ~ " # Rough terrain
            else:
                line += " . "
        print(line)

In [2]:
# --- EXPERIMENT 1: Initial Grid ---
print("--- Initial Grid Problem ---")

# Define the grid based on the problem statement
# 0 = empty, 1 = obstacle (#)
# Row 5 is rough terrain (cost = 3)
initial_grid = [[0] * 10 for _ in range(10)]

# Set obstacles [cite: 26]
# row 3, cols 2-5
for c in range(2, 6): initial_grid[2][c] = 1
# row 6, cols 4-7
for c in range(4, 8): initial_grid[5][c] = 1 # Correcting problem statement (row 6 should be index 5)
for c in range(2, 6): initial_grid[3][c] = 1

start = (0, 0) # [cite: 24]
goal = (9, 9)   # [cite: 25]

# Run A* on the initial grid
path, cost, expanded = a_star_search(initial_grid, start, goal)

print("\nInitial Grid Visualization with Path (*):")
print_grid_with_path(initial_grid, path)

print(f"\nPath Sequence: {path}")
print(f"Path Cost: {cost}")
print(f"Number of Nodes Expanded: {expanded}\n")

--- Initial Grid Problem ---

Initial Grid Visualization with Path (*):
 S  *  *  *  .  .  .  .  .  . 
 .  .  .  *  *  *  *  *  .  . 
 .  .  #  #  #  #  .  *  *  . 
 .  .  #  #  #  #  .  .  *  * 
 .  .  .  .  .  .  .  .  .  * 
 ~  ~  ~  ~  #  #  #  #  ~  * 
 .  .  .  .  .  .  .  .  .  * 
 .  .  .  .  .  .  .  .  .  * 
 .  .  .  .  .  .  .  .  .  * 
 .  .  .  .  .  .  .  .  .  G 

Path Sequence: [(0, 0), (0, 1), (0, 2), (0, 3), (1, 3), (1, 4), (1, 5), (1, 6), (1, 7), (2, 7), (2, 8), (3, 8), (3, 9), (4, 9), (5, 9), (6, 9), (7, 9), (8, 9), (9, 9)]
Path Cost: 20
Number of Nodes Expanded: 74



In [3]:
# --- EXPERIMENT 2: Modified Grid ---
print("--- Modified Grid Problem ---")

# Modify the grid by blocking cell (4, 4)
modified_grid = [row[:] for row in initial_grid] # Create a copy
modified_grid[4][4] = 1

# Rerun A* on the modified grid
path_mod, cost_mod, expanded_mod = a_star_search(modified_grid, start, goal)

print("\nModified Grid Visualization with Path (*):")
print_grid_with_path(modified_grid, path_mod)

print(f"\nPath Sequence: {path_mod}")
print(f"Path Cost: {cost_mod}")
print(f"Number of Nodes Expanded: {expanded_mod}")

--- Modified Grid Problem ---

Modified Grid Visualization with Path (*):
 S  *  .  .  .  .  .  .  .  . 
 .  *  .  .  .  .  .  .  .  . 
 .  *  #  #  #  #  .  .  .  . 
 .  *  #  #  #  #  .  .  .  . 
 .  *  *  *  #  .  .  .  .  . 
 ~  ~  ~  *  #  #  #  #  ~  ~ 
 .  .  .  *  .  .  .  .  .  . 
 .  .  .  *  .  .  .  .  .  . 
 .  .  .  *  *  .  .  .  .  . 
 .  .  .  .  *  *  *  *  *  G 

Path Sequence: [(0, 0), (0, 1), (1, 1), (2, 1), (3, 1), (4, 1), (4, 2), (4, 3), (5, 3), (6, 3), (7, 3), (8, 3), (8, 4), (9, 4), (9, 5), (9, 6), (9, 7), (9, 8), (9, 9)]
Path Cost: 20
Number of Nodes Expanded: 80
